<a href="https://colab.research.google.com/github/mafemunoz/Trabajo-IA-2025-2/blob/main/03%20-%20modelo%20con%20preprocesado%20por%20moda%20y%20red%20neuronal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Carga el archivo kaggle.json
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!python -m pip install -qq kaggle
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia
!unzip udea-ai-4-eng-20252-pruebas-saber-pro-colombia

Saving kaggle (2).json to kaggle (2) (1).json
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 4, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.12/dist-packages/kaggle/__init__.py", line 6, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 434, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/
unzip:  cannot find or open udea-ai-4-eng-20252-pruebas-saber-pro-colombia, udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip or udea-ai-4-eng-20252-pruebas-saber-pro-colombia.ZIP.


In [7]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [8]:
test_df = pd.read_csv('/content/test.csv', encoding= 'utf-8')
df = pd.read_csv('/content/train.csv', encoding= 'utf-8')

FileNotFoundError: [Errno 2] No such file or directory: '/content/test.csv'

Preprocesado
El preprocesamiento implementado resultó ser muy efectivo, manteniendo la limpieza de la columna PROGRAMA como en versiones anteriores. En este caso, las columnas categóricas y numéricas con valores faltantes fueron completadas utilizando la moda de cada una. Además, se eliminaron columnas irrelevantes y duplicadas como E_PRIVADO_LIBERTAD y la repetida F_TIENEINTERNET.1. Para facilitar su manipulación, se renombraron las columnas, se convirtieron los valores booleanos a binarios, y se aplicó codificación One-Hot a todas las variables categóricas seleccionadas. Finalmente, la variable RENDIMIENTO_GLOBAL fue transformada a un valor ordinal, permitiendo su uso en modelos supervisados que requieren variables numéricas.

In [ ]:
category_cols = [i for i in df.columns if not i in df._get_numeric_data()]
for c in category_cols:
    print ("%10s"%c, np.unique(df[c].dropna()))


def Preprocesar(df):
  def obtener_primera_frase(cadena):
    return cadena.split(' ', 1)[0]

  def quitar_tildes(palabra):
    return re.sub(r'[ÁÉÍÓÚ]', lambda x: {'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U'}[x.group()], palabra)

  df['E_PRGM_ACADEMICO'] = df['E_PRGM_ACADEMICO'].apply(obtener_primera_frase)
  df['E_PRGM_ACADEMICO'] = df['E_PRGM_ACADEMICO'].apply(quitar_tildes)

  reemplazos = {
      'ADMINISTRACION': ['ADMINISTRACION', 'ADMINISTRACI¿N', 'ADMINISTRACIÒN', 'ADMINISTRACIÓN', 'ADMINSITRACION', 'ADMINSITRACIÓN'],
      'ANTROPOLOGIA': ['ANTROPOLOGIA', 'ANTROPOLOGÍA'],
      'ARTES': ['ARTE', 'ARTES','BELLAS','ANIMACION'],
      'BIOINGENIERIA': ['BIOINGENIERIA', 'BIOINGENIERÍA'],
      'BIOLOGIA': ['BIOLOGIA', 'BIOLOGÍA'],
      'COMUNICACION': ['COMUNICACION', 'COMUNICACIONES', 'COMUNICACI¿N', 'COMUNICACIÓN', 'COMUNICACIÖN'],
      'CONSTRUCCION': ['CONSTRUCCION', 'CONSTRUCCIONES'],
      'CONTADURIA': ['CONTADURIA', 'CONTADURÍA'],
      'DISEÑO': ['DISENO', 'DISE¿O', 'DISE¿¿O', 'DISEÑO'],
      'ECOLOGIA': ['ECOLOGIA', 'ECOLOGÍA'],
      'ECONOMIA': ['ECONOM¿A'],
      'GEOLOGIA': ['GEOLOG¿A'],
      'GERENCIA': ['GERONTOLOGIA'],
      'INGENIERIA': ['INGENIER¿A', 'INGENIERÌA', 'INGENIIERIA','AGROINDUSTRIA'],
      'INSTRUMENTACION': ['INTRUMENTACION'],
      'MATEMATICAS': ['MATEMATICA'],
      'MARKETING': ['MERCADEO', 'MERCADOLOGIA'],
      'PSICOLOGIA': ['PSICOLOGA'],
      'QUIMICA': ['QUIMICA.', 'QU¿MICA'],
      'MUSICA': ['MUSICA-'],
      'TERAPIAS': ['TERAPIA', 'TERAPIAS'],
      'GESTION': ['GESTI¿N'],
      'NEGOCIOS' : ['3°'],
      'DEPORTE' : ['ACTIVIDAD'],
      'LICENCIATURA' : ['Licenciatura','LICENC.EN'],
      'LENGUAS' :['LENGUAJES']

  }

  for key, values in reemplazos.items():
      df['E_PRGM_ACADEMICO'] = df['E_PRGM_ACADEMICO'].replace(values, key)

  ##Eliminar columnas

  df = df.drop(labels =['E_PRIVADO_LIBERTAD','F_TIENEINTERNET.1', 'INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4'],axis= 1)

  if 'RENDIMIENTO_GLOBAL' in df.columns:
    df.columns = ['ID','PERIODO','PROGRAMA','DEPARTAMENTO','VALOR_MATRICULA','HORAS_TRABAJA','ESTRATO','TIENE_INTERNET','EDUCACION_PADRE','TIENE_LAVADORA','TIENE_AUTO','PAGA_MATRICULA','TIENE_COMPUTADOR','EDUCACION_MADRE','RENDIMIENTO_GLOBAL']
  else:
    df.columns = ['ID','PERIODO','PROGRAMA','DEPARTAMENTO','VALOR_MATRICULA','HORAS_TRABAJA','ESTRATO','TIENE_INTERNET','EDUCACION_PADRE','TIENE_LAVADORA','TIENE_AUTO','PAGA_MATRICULA','TIENE_COMPUTADOR','EDUCACION_MADRE']

  ##Convertir a binario

  def convertir_num(valor):
      if valor == 'Si':
          return 1
      elif valor == 'No':
          return 0
      else:
          return valor

  # Aplicar la función a todas las columnas del DataFrame
  df = df.applymap(convertir_num)

  ##Imputar Datos

  df['ESTRATO'] = df['ESTRATO'].str.replace('Estrato ', '')

  columnas_binarias = ['TIENE_INTERNET', 'TIENE_LAVADORA', 'TIENE_AUTO',
                        'PAGA_MATRICULA', 'TIENE_COMPUTADOR']

  for columna in columnas_binarias:
      moda = df[columna].mode()[0]
      df[columna].fillna(moda, inplace=True)


  ModaEstrato = df['ESTRATO'].mode()[0]  # Obtener la moda
  df['ESTRATO'].fillna(ModaEstrato, inplace=True)

  ModaPadre = df['EDUCACION_PADRE'].mode()[0]
  df['EDUCACION_PADRE'].fillna(ModaPadre, inplace=True)

  ModaMadre = df['EDUCACION_MADRE'].mode()[0]
  df['EDUCACION_MADRE'].fillna(ModaMadre, inplace=True)

  ValorMatricula = df['VALOR_MATRICULA'].mode()[0]
  df['VALOR_MATRICULA'].fillna(ValorMatricula, inplace=True)

  HorasTrabaja = df['HORAS_TRABAJA'].mode()[0]
  df['HORAS_TRABAJA'].fillna(HorasTrabaja, inplace=True)

  ##Convertir columnas categoricas
  Columnas_categoricas = ["VALOR_MATRICULA","PROGRAMA","DEPARTAMENTO","EDUCACION_MADRE","EDUCACION_PADRE"]

  df[Columnas_categoricas] = df[Columnas_categoricas].astype("category")

  valores_rendimiento = {'alto': 4,'medio-alto': 3,'medio-bajo':2, 'bajo': 1}
  df= df.replace(valores_rendimiento)

  ##Convertir a Onehot
  encoder = OneHotEncoder(sparse_output=False)

  encoded_columns = encoder.fit_transform(df[['HORAS_TRABAJA','PERIODO','VALOR_MATRICULA','PROGRAMA','DEPARTAMENTO','ESTRATO','EDUCACION_PADRE','EDUCACION_MADRE']])

  feature_names = encoder.get_feature_names_out(['HORAS_TRABAJA','PERIODO', 'VALOR_MATRICULA','PROGRAMA','DEPARTAMENTO','ESTRATO','EDUCACION_PADRE','EDUCACION_MADRE'])

  one_hot_encoded_df = pd.DataFrame(encoded_columns, columns=feature_names)

  df = pd.concat([df, one_hot_encoded_df], axis=1)

  df = df.drop(labels =['HORAS_TRABAJA', 'PERIODO','VALOR_MATRICULA','PROGRAMA','DEPARTAMENTO','ESTRATO','EDUCACION_PADRE','EDUCACION_MADRE'],axis= 1)

  return df

E_PRGM_ACADEMICO ['3°  CICLO PROFESIONAL NEGOCIOS INTERNACIONALES'
 'ACTIVIDAD FISICA Y DEPORTE' 'ACUICULTURA' 'ADMINISTRACION'
 'ADMINISTRACION  FINANCIERA' 'ADMINISTRACION & SERVICIO'
 'ADMINISTRACION AERONAUTICA' 'ADMINISTRACION AGROPECUARIA'
 'ADMINISTRACION AMBIENTAL'
 'ADMINISTRACION AMBIENTAL Y DE LOS RECURSOS NATURALES'
 'ADMINISTRACION BANCARIA Y FINANCIERA' 'ADMINISTRACION COMERCIAL'
 'ADMINISTRACION COMERCIAL Y DE MERCADEO'
 'ADMINISTRACION DE COMERCIO EXTERIOR' 'ADMINISTRACION DE EMPRESAS'
 'ADMINISTRACION DE EMPRESAS  Y  GESTION AMBIENTAL'
 'ADMINISTRACION DE EMPRESAS AGROINDUSTRIALES'
 'ADMINISTRACION DE EMPRESAS AGROPECUARIAS'
 'ADMINISTRACION DE EMPRESAS COMERCIALES'
 'ADMINISTRACION DE EMPRESAS EN TELECOMUNICACIONES'
 'ADMINISTRACION DE EMPRESAS TURISTICA'
 'ADMINISTRACION DE EMPRESAS TURISTICAS'
 'ADMINISTRACION DE EMPRESAS TURISTICAS Y HOTELERAS'
 'ADMINISTRACION DE EMPRESAS TURÍSTICAS Y HOTELERAS'
 'ADMINISTRACION DE EMPRESAS Y FINANZAS'
 'ADMINISTRACION DE GESTISN 

In [ ]:
df = Preprocesar(df)
test_df = Preprocesar(test_df)

/tmp/ipython-input-2550013136.py:69: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(convertir_num)
/tmp/ipython-input-2550013136.py:80: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[columna].fillna(moda, inplace=True)
/tmp/ipython-input-2550013136.py:84: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the in

In [ ]:
X = df.drop(labels=["RENDIMIENTO_GLOBAL", "ID"], axis=1)  # Características
y = df["RENDIMIENTO_GLOBAL"]  # Variable objetivo

# Codificar las etiquetas de clase
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Número de clases en y
num_classes = len(label_encoder.classes_)

model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))  # Número de unidades igual a clases y softmax para clasificación multiclase

# Compilar el modelo
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print("Accuracy:", accuracy)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
17313/17313 ━━━━━━━━━━━━━━━━━━━━ 60s 3ms/step - accuracy: 0.4013 - loss: 1.2513 - val_accuracy: 0.4232 - val_loss: 1.2180
Epoch 2/10
17313/17313 ━━━━━━━━━━━━━━━━━━━━ 60s 3ms/step - accuracy: 0.4199 - loss: 1.2249 - val_accuracy: 0.4257 - val_loss: 1.2144
Epoch 3/10
17313/17313 ━━━━━━━━━━━━━━━━━━━━ 62s 4ms/step - accuracy: 0.4234 - loss: 1.2201 - val_accuracy: 0.4274 - val_loss: 1.2112
Epoch 4/10
17313/17313 ━━━━━━━━━━━━━━━━━━━━ 59s 3ms/step - accuracy: 0.4241 - loss: 1.2193 - val_accuracy: 0.4268 - val_loss: 1.2132
Epoch 5/10
17313/17313 ━━━━━━━━━━━━━━━━━━━━ 64s 4ms/step - accuracy: 0.4251 - loss: 1.2166 - val_accuracy: 0.4281 - val_loss: 1.2104
Epoch 6/10
17313/17313 ━━━━━━━━━━━━━━━━━━━━ 58s 3ms/step - accuracy: 0.4243 - loss: 1.2157 - val_accuracy: 0.4277 - val_loss: 1.2141
Epoch 7/10
17313/17313 ━━━━━━━━━━━━━━━━━━━━ 60s 3ms/step - accuracy: 0.4259 - loss: 1.2158 - val_accuracy: 0.4284 - val_loss: 1.2096
Epoch 8/10
17313/17313 ━━━━━━━━━━━━━━━━━━━━ 64s 4ms/step - accuracy: 

In [ ]:
predictions = model.predict(test_df.drop('ID', axis=1))

predicted_classes = tf.argmax(predictions, axis=1) + 1  # Sumar 1 para ajustar el índice de las clases

submit = pd.DataFrame({'ID': test_df['ID'], 'RENDIMIENTO_GLOBAL': predicted_classes})

# Mapear las clases predichas al rango de etiquetas originales
submit['RENDIMIENTO_GLOBAL'] = submit['RENDIMIENTO_GLOBAL'].replace({
    1: 'bajo',
    2: 'medio-bajo',
    3: 'medio-alto',
    4: 'alto'
})

submit.to_csv('submission.csv', index=False)

9275/9275 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step
